<div>
    <img src="img/AF-logotiposecundario-GiantSteps-positivo-2.png" width="250px">
</div>

<h2>The Question:</h2>
<span>What was the most profitable period of 500 calendar days since 2000-01-01 until 2022-03-31? That is, if you had to invest C amount of capital for 500 days, what would have been the most profitable period from the beginning of 2000 until the end of March/2022?</span>

<h2>The Answer:</h2>

In [8]:
import pandas as pd, urllib.request, json, warnings

warnings.filterwarnings('ignore')
std_date_format = '%d/%m/%Y'
profit_period = 500

In [9]:
START_DATE_FINAL = '01/01/2000'
END_DATE_FINAL = '31/03/2021'

initial_day = START_DATE_FINAL.split('/')[0]
initial_month = START_DATE_FINAL.split('/')[1]
initial_year = START_DATE_FINAL.split('/')[2]

final_day = END_DATE_FINAL.split('/')[0]
final_month = END_DATE_FINAL.split('/')[1]
final_year = END_DATE_FINAL.split('/')[2]

In [10]:
selic_rates_url = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.11/dados?formato=json&dataInicial={initial_day}/{initial_month}/{initial_year}&dataFinal={final_day}/{final_month}/{final_year}'

with urllib.request.urlopen(selic_rates_url) as url:
    data = json.loads(url.read().decode())
    selic_rates = pd.DataFrame(data)

selic_rates['data'] = pd.to_datetime(selic_rates['data'], format=std_date_format, dayfirst=True)
selic_rates = selic_rates.loc[(selic_rates['data'].between(START_DATE_FINAL, END_DATE_FINAL))]
selic_rates.style.format({'data': lambda t: t.strftime(std_date_format)})
selic_rates['valor'] = pd.to_numeric(selic_rates['valor'])
selic_rates['valor_ajustado'] = selic_rates['valor']/100

In [11]:
sum_df = selic_rates['valor_ajustado'].rolling(profit_period).sum()
idxmax = sum_df.idxmax()

In [12]:
best_period = selic_rates[selic_rates.index.isin(range((idxmax - 499), (idxmax + 1)))]
best_period

,data,valor,valor_ajustado
458,2001-10-29,0.069220,0.000692
459,2001-10-30,0.069220,0.000692
460,2001-10-31,0.069220,0.000692
461,2001-11-01,0.069220,0.000692
462,2001-11-05,0.069220,0.000692
...,...,...,...
953,2003-10-16,0.071846,0.000718
954,2003-10-17,0.071846,0.000718
955,2003-10-20,0.071846,0.000718
956,2003-10-21,0.071846,0.000718


In [13]:
initial_best_date = best_period['data'].dt.strftime(std_date_format).values[0]
final_best_date = best_period['data'].dt.strftime(std_date_format).values[-1]

rates_list = best_period['valor_ajustado'].tolist()
total_rate = 1

for r in rates_list:
    total_rate *= (1 + r)
    
increase = round(((total_rate - 1) * 100), 2)

In [14]:
print(f'The most profitable period of {profit_period} calendar days between {START_DATE_FINAL} and {END_DATE_FINAL} was between {initial_best_date} and {final_best_date}, where an investment would have yielded an increase of approximately {increase}%.')

The most profitable period of 500 calendar days between 01/01/2000 and 31/03/2021 was between 29/10/2001 and 22/10/2003, where an investment would have yielded an increase of approximately 46.7%.
